In [2]:
%load_ext autoreload
%autoreload 2
from pymongo import MongoClient
import sys
from pathlib import Path
from tqdm import tqdm
import json
from bson import ObjectId
from pprint import pprint
from datetime import datetime


sys.path.append(str(Path("..").resolve()))
from src import *

# Update Queries
ℹ️ This notebook require the execution of [`1.3-Consistency_Recover`](../1-migration/1.3-Consistency_Recover.ipynb). Or you can just load the snapshop as in the following cell.

## Load post-preprocessing data
The following cell reset the `omero_museum` database loading the `4_consistent` snapshot.

In [ ]:
%%capture
MongoClient("mongodb://localhost:27017/").drop_database("omero_museum")
!mongorestore --host localhost:27017 --drop --db omero_museum  "../backup/5_optimized/omero_museum"

In [3]:
connector = MongoDBConnector("omero_museum", silent=True)
db = connector.db

### UP1: _Update the description of a material_

In [6]:
old_name = "Clay"
new_name = "White Clay"

db.artworks.update_many(
    {"materials": old_name},
    {"$set": {"materials.$[m]": new_name}},
    array_filters=[{"m": old_name}],
)

UpdateResult({'n': 2, 'nModified': 2, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

### UP2: _Check-in a visitor's ticket_

In [16]:
ticket_date = datetime(2025, 9, 2, 13, 7, 37)
visitor = db.visitors.find_one({"tickets": {"$ne": None}, "tickets.0.check_in": False})
visitor_id = visitor["_id"]  # type: ignore
ticket_id = visitor["tickets"][0]["_id"]  # type: ignore

db.visitors.update_one(
    {"_id": visitor_id},
    {"$set": {"tickets.$[t].check_in": True}},
    array_filters=[{"t._id": ticket_id}],
)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

### UP3: _Update trade information of an artwork_

In [ ]:
customer_id = db.visitors.find_one({"is_customer": True})["_id"]  # type: ignore

db.artworks.update_one(
    {"_id": "La Notte Stellata"}, 
    {"$set": {
        "trade.win_price": 5000,
        "trade.winner_id": customer_id,
    }}
)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

### UP4: _Update information of an artwork_

In [12]:
db.artworks.update_one(
    {"_id": "La Notte Stellata"}, 
    {"$set": {
        "isOriginal": False,
        "tecniques": ["painting"],
        "materials": ["Glass", "Bronze"],
    }}
)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

### UP5: _Update curriculum of an employee_

In [21]:
employee_id = db.roles.find_one()["_id"]  # type: ignore
new_curriculum = "https://linkedin.com/in/matilda-scerbo"

db.employees.update_one(
    {"_id": employee_id}, {"$set": {"curriculum": new_curriculum}}
)

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

### UP6: _Update employee's department_

In [22]:
employee_id = db.roles.find_one()["_id"]  # type: ignore
department_id = db.departments.find_one()["_id"]  # type: ignore

db.employees.update_one({"_id": employee_id}, {"$set": {"department": department_id}})

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

### UP7: _Update the shift of an given employee_

In [28]:
employee_id = db.roles.find_one()["_id"]  # type: ignore
day_of_week = "sabato"
new_time_start = "13:00:00"
new_time_end = "18:30:00"

db.roles.update_one(
    {"_id": employee_id},
    {
        "$set": {
            "shifts.$[s].time_start": new_time_start,
            "shifts.$[s].time_end": new_time_end,
        }
    },
    array_filters=[{"s.day_of_week": day_of_week}],
)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

### UP8: _Change the date of a limited event_

In [ ]:
limited_event_id = "The Future of Mosaic"

db.limited_events.update_one(
    {"_id": limited_event_id},
    {
        "$set": {
            "startDate": datetime(2025, 10, 2, 10, 0),
            "endDate": datetime(2025, 10, 2, 18, 0),
        }
    },
)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

### UP9: _Update a message in a given chat_

In [31]:
message_id = db.visitors.find_one({"chat.old_messages": {"$exists": True}})["chat"]["old_messages"][0]  # type: ignore
new_content = "A new message content"

db.messages.update_one({"_id": message_id}, {"$set": {"content": new_content}})

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

### UP10: _Update prices of a workshop_

In [ ]:
workshop_id = "Pittura su tela"

db.workshops.update_one(
    {"_id": workshop_id}, {"$set": {"price_person": 30.00, "price_class": 90.00}}
)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)